In [60]:
from random import randint

class Ship:
    def __init__(self, length, tp=1, x=None, y=None) -> None:
        self._x = x
        self._y = y
        self._length = length
        self._tp = tp
        self._is_move = True
        self._cells = [1 for _ in range(self._length)]

    def set_start_coords(self, x, y):
        self._x = x
        self._y = y

    def get_start_coords(self):
        return self._x, self._y

    def is_collide(self, ship):
        if self._tp == 1 and ship._tp == 1:
            return True if (self._x + self._length - 1 >= ship._x -1 and \
                            self._x <= ship._x + ship._length) and ship._y - 1 <= self._y <= ship._y + 1 else False

        if self._tp == 2 and ship._tp == 2:
            return True if (self._y + self._length - 1 >= ship._y -1 and \
                            self._y <= ship._y + ship._length) and ship._x - 1 <= self._x <= ship._x + 1 else False

        if self._tp == 1 and ship._tp == 2:
            return True if (self._x + self._length - 1 >= ship._x -1 and \
                            self._x <= ship._x + 1) and ship._y - 1 <= self._y <= ship._y + ship._length else False

        if self._tp == 2 and ship._tp == 1:
            return True if (ship._x + ship._length - 1 >= self._x -1 and \
                            ship._x <= self._x + 1) and self._y - 1 <= ship._y <= self._y + self._length else False

    def is_out_pole(self, size):
        if 0 <= self._x <= size - 1 and 0 <= self._y <= size - 1 and \
            0 <= self._x + self._length - 1 <= size - 1 and \
            self._tp == 1:
            return False

        if 0 <= self._x <= size - 1 and 0 <= self._y <= size - 1 and \
            0 <= self._y + self._length - 1 <= size - 1 and \
            self._tp == 2:
            return False

        return True

    def move(self, go):
        if self._tp == 1 and self._is_move:
            self._x += go
        
        if self._tp == 2 and self._is_move:
            self._y += go

    def __getitem__(self, index):
        return self._cells[index]

    def __setitem__(self, index, value):
        self._cells[index] = value

class GamePole:
    def __init__(self, size) -> None:
        self._size = size
        self._ships = []

    def init(self):
        self._ships = [
                        Ship(4, tp=randint(1, 2)), Ship(3, tp=randint(1, 2)), Ship(3, tp=randint(1, 2)),
                        Ship(2, tp=randint(1, 2)), Ship(2, tp=randint(1, 2)), Ship(2, tp=randint(1, 2)),
                        Ship(1, tp=randint(1, 2)), Ship(1, tp=randint(1, 2)), Ship(1, tp=randint(1, 2)),
                        Ship(1, tp=randint(1, 2)), 
        ]

        for i in range(len(self._ships)):
            ship = self._ships[i]
            while True:
                ship.set_start_coords(randint(0, self._size), randint(0, self._size))

                collide = []
                for other_ship in self._ships[:i]:
                    collide.append(ship.is_collide(other_ship))
                if len(collide) == 0:
                    collide.append(False)
                    
                if not ship.is_out_pole(self._size) and not any(collide):
                    break

    def get_ships(self):
        return self._ships

    def move_ships(self):
        pass
    
    def show(self):
        for y in self.get_pole():
            print(*y)

    def get_pole(self):
        pole =  [[0 for x in range(self._size)] for y in range(self._size)]
        
        for ship in self._ships:
            x, y = ship.get_start_coords()
            labels = ship._cells

            if ship._tp == 1:
                for i in range(ship._length):
                    pole[y][x + i] = ship._cells[i]
            
            if ship._tp == 2:
                for i in range(ship._length):
                    pole[y + i][x] = ship._cells[i]

        return tuple(tuple(y) for y in pole)

In [61]:
ship = Ship(2)
ship = Ship(2, 1)
ship = Ship(3, 2, 0, 0)

assert ship._length == 3 and ship._tp == 2 and ship._x == 0 and ship._y == 0, "неверные значения атрибутов объекта класса Ship"
assert ship._cells == [1, 1, 1], "неверный список _cells"
assert ship._is_move, "неверное значение атрибута _is_move"

ship.set_start_coords(1, 2)
assert ship._x == 1 and ship._y == 2, "неверно отработал метод set_start_coords()"
assert ship.get_start_coords() == (1, 2), "неверно отработал метод get_start_coords()"

ship.move(1)
s1 = Ship(4, 1, 0, 0)
s2 = Ship(3, 2, 0, 0)
s3 = Ship(3, 2, 0, 2)

assert s1.is_collide(s2), "неверно работает метод is_collide() для кораблей Ship(4, 1, 0, 0) и Ship(3, 2, 0, 0)"
assert s1.is_collide(s3) == False, "неверно работает метод is_collide() для кораблей Ship(4, 1, 0, 0) и Ship(3, 2, 0, 2)"

s2 = Ship(3, 2, 1, 1)
assert s1.is_collide(s2), "неверно работает метод is_collide() для кораблей Ship(4, 1, 0, 0) и Ship(3, 2, 1, 1)"

s2 = Ship(3, 1, 8, 1)
assert s2.is_out_pole(10), "неверно работает метод is_out_pole() для корабля Ship(3, 1, 8, 1)"

s2 = Ship(3, 2, 1, 5)
assert s2.is_out_pole(10) == False, "неверно работает метод is_out_pole(10) для корабля Ship(3, 2, 1, 5)"

s2[0] = 2
assert s2[0] == 2, "неверно работает обращение ship[indx]"

p = GamePole(10)
p.init()
for nn in range(5):
    for s in p._ships:
        assert s.is_out_pole(10) == False, "корабли выходят за пределы игрового поля"

        for ship in p.get_ships():
            if s != ship:
                assert s.is_collide(ship) == False, "корабли на игровом поле соприкасаются"
    p.move_ships()

gp = p.get_pole()
assert type(gp) == tuple and type(gp[0]) == tuple, "метод get_pole должен возвращать двумерный кортеж"
assert len(gp) == 10 and len(gp[0]) == 10, "неверные размеры игрового поля, которое вернул метод get_pole"

pole_size_8 = GamePole(8)
pole_size_8.init()
